In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, Callback

# Load data set

In [ ]:
tmp_x = np.load('MinMax_scaled_x.npy') # MinMax-scaled x coordinates
tmp_y = np.load('MinMax_scaled_y.npy') # MinMax-scaled x coordinates
y = np.load('log_Lp.npy') # Target value (Lp)
Lw = tmp_x.shape[1] # Window size

# Construct training & validation data sets

In [ ]:
x = np.zeros((tmp_x.shape[0],tmp_x.shape[1],2)) # Input data
for i in range(x.shape[0]) :
    x[i,:,0] = tmp_x[i]
    x[i,:,1] = tmp_y[i]

In [ ]:
# Split the data set into training and validation data sets
n_samples = x.shape[0]
IDX = np.arange(0,n_samples,1,dtype = 'int')
np.random.shuffle(IDX)
train_x = x[IDX[:int(n_samples*0.9)]]
test_x = x[IDX[int(n_samples*0.9):]]
train_y = y[IDX[:int(n_samples*0.9)]]
test_y = y[IDX[int(n_samples*0.9):]]

# Implement transformer-based architecture

In [ ]:
# NOTE
This code provides the architectural skeleton of our Transformer-based model used in the manuscript.
The detailed implementation is currently withheld and will be released upon acceptance.

- `TransformerBlock`: Multi-head attention and feedforward layers (details hidden)
- `PositionalEncoding`: Trainable positional embedding (details hidden)
- `build_transformer_model`: Builds the full Keras model (hidden)

For review purposes, a minimal subset of code may be shared upon request.

class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, dropout_rate=0.1):
        super(TransformerBlock, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.dropout_rate = dropout_rate
        raise NotImplementedError("TransformerBlock implementation is withheld during the preprint stage.")

    def call(self, inputs, training=False):
        raise NotImplementedError("TransformerBlock forward pass is withheld during the preprint stage.")


class PositionalEncoding(layers.Layer):
    def __init__(self, sequence_length, embed_dim):
        super(PositionalEncoding, self).__init__()
        self.sequence_length = sequence_length
        self.embed_dim = embed_dim
        raise NotImplementedError("PositionalEncoding implementation is withheld during the preprint stage.")

    def call(self, inputs):
        raise NotImplementedError("PositionalEncoding call method is withheld during the preprint stage.")


def build_transformer_model(sequence_length, embed_dim=64, num_heads=64, ff_dim=128, num_layers=5, dropout_rate=0):
    """
    Build a Transformer-based regression model for sequence input of shape (batch_size, sequence_length, 2).
    Actual implementation of TransformerBlock and PositionalEncoding is omitted.
    """
    raise NotImplementedError("Model construction is withheld during the preprint stage.")

In [ ]:
X = build_transformer_model(Lw)
X.compile(optimizer = Adam(10**-4), loss = 'MAE')
X.summary()

# Training setting

In [ ]:
MODEL_SAVE_FOLDER_PATH = './'
model_path = MODEL_SAVE_FOLDER_PATH + 'Model.hdf5'
cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss', mode = 'min', verbose=1, save_best_only=True) # Model checkpoint
cb_early_stopping = EarlyStopping(monitor='val_loss', mode = 'min', patience=50, restore_best_weights=True) # Early stopping
lr_reduction = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=10**-0.5, patience=15, min_lr=1e-7) # Reduce learning rate

In [ ]:
history = X.fit(train_x,train_y, epochs=10000,batch_size = 2**10, callbacks=[cb_checkpoint,cb_early_stopping,SaveHistory(),lr_reduction],validation_data=(test_x,test_y))